In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['figure.figsize'] = (10,7)
plt.rcParams['figure.dpi'] = (150)
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_squared_error,classification_report
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression
sns.set_theme(style='darkgrid',palette='rainbow')
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
df=pd.read_csv('C:\\Users\\Abhishek\\Downloads\\Fraud_check.csv',encoding='latin1')
df

,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban
0,NO,Single,68833,50047,10,YES
1,YES,Divorced,33700,134075,18,YES
2,NO,Married,36925,160205,30,YES
3,YES,Single,50190,193264,15,YES
4,NO,Married,81002,27533,28,NO
...,...,...,...,...,...,...
595,YES,Divorced,76340,39492,7,YES
596,YES,Divorced,69967,55369,2,YES
597,NO,Divorced,47334,154058,0,YES
598,YES,Married,98592,180083,17,NO


In [3]:
df.describe()

,Taxable.Income,City.Population,Work.Experience
count,600.000000,600.000000,600.000000
mean,55208.375000,108747.368333,15.558333
std,26204.827597,49850.075134,8.842147
min,10003.000000,25779.000000,0.000000
25%,32871.500000,66966.750000,8.000000
50%,55074.500000,106493.500000,15.000000
75%,78611.750000,150114.250000,24.000000
max,99619.000000,199778.000000,30.000000


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Undergrad        600 non-null    object
 1   Marital.Status   600 non-null    object
 2   Taxable.Income   600 non-null    int64 
 3   City.Population  600 non-null    int64 
 4   Work.Experience  600 non-null    int64 
 5   Urban            600 non-null    object
dtypes: int64(3), object(3)
memory usage: 28.2+ KB


In [6]:
from sklearn.preprocessing import  OrdinalEncoder
oe=OrdinalEncoder()
char_col=df.select_dtypes('object').columns
df[char_col]=oe.fit_transform(df[char_col])
df.head()

,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban
0,0.0,2.0,68833,50047,10,1.0
1,1.0,0.0,33700,134075,18,1.0
2,0.0,1.0,36925,160205,30,1.0
3,1.0,2.0,50190,193264,15,1.0
4,0.0,1.0,81002,27533,28,0.0


In [7]:
#taxable_income <= 30000 ---risky
target=[]
for i in df['Taxable.Income']:
    if i<=30000:
        target.append('Risky')
    else:
        target.append('Good')
        
df['tax_income']=target 
df.drop(columns='Taxable.Income',inplace=True)
df

,Undergrad,Marital.Status,City.Population,Work.Experience,Urban,tax_income
0,0.0,2.0,50047,10,1.0,Good
1,1.0,0.0,134075,18,1.0,Good
2,0.0,1.0,160205,30,1.0,Good
3,1.0,2.0,193264,15,1.0,Good
4,0.0,1.0,27533,28,0.0,Good
...,...,...,...,...,...,...
595,1.0,0.0,39492,7,1.0,Good
596,1.0,0.0,55369,2,1.0,Good
597,0.0,0.0,154058,0,1.0,Good
598,1.0,1.0,180083,17,0.0,Good


In [9]:
x=df.iloc[:,:-1]
y=df['tax_income']

In [10]:
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.25,random_state=2)

In [11]:
def fun(model):
    re=model
    re.fit(xtrain,ytrain)
    ypred=re.predict(xtest)
    print(classification_report(ytest,ypred))
    print(confusion_matrix(ytest,ypred))
    print(f'tranning score {re.score(xtrain,ytrain)}')
    print(f'testing score {re.score(xtest,ytest)}')

In [13]:
from sklearn.metrics import classification_report,confusion_matrix
fun(RandomForestClassifier())

              precision    recall  f1-score   support

        Good       0.78      0.95      0.86       117
       Risky       0.25      0.06      0.10        33

    accuracy                           0.75       150
   macro avg       0.52      0.50      0.48       150
weighted avg       0.66      0.75      0.69       150

[[111   6]
 [ 31   2]]
tranning score 1.0
testing score 0.7533333333333333


In [14]:
# Hyperparameter Tuning

In [15]:
params={
    'criterion':['gini','entropy'],
    'max_depth':[1,2,4,6,8,10,13],
    'max_features':['auto','sqrt','log'],
    'min_samples_split':[4,6,8,12,9]
}

In [17]:
from sklearn.model_selection import train_test_split,GridSearchCV

grid=GridSearchCV(RandomForestClassifier(n_estimators=150),param_grid=params,verbose=-1)

In [18]:
grid.fit(xtrain,ytrain)

GridSearchCV(estimator=RandomForestClassifier(n_estimators=150),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [1, 2, 4, 6, 8, 10, 13],
                         'max_features': ['auto', 'sqrt', 'log'],
                         'min_samples_split': [4, 6, 8, 12, 9]},
             verbose=-1)

In [19]:
grid.best_params_

{'criterion': 'gini',
 'max_depth': 1,
 'max_features': 'auto',
 'min_samples_split': 4}

In [20]:
fun(RandomForestClassifier(n_estimators=150,criterion='gini',max_depth=1,max_features='auto',min_samples_split=4))

              precision    recall  f1-score   support

        Good       0.78      1.00      0.88       117
       Risky       0.00      0.00      0.00        33

    accuracy                           0.78       150
   macro avg       0.39      0.50      0.44       150
weighted avg       0.61      0.78      0.68       150

[[117   0]
 [ 33   0]]
tranning score 0.7977777777777778
testing score 0.78
